In [10]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
path = str(Path.cwd().parent)
print(path)
sys.path.insert(1, path)

import numpy as np
import pandas as pd
import skforecast

print(skforecast.__version__)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
c:\Users\jaesc2\GitHub\skforecast
0.20.0


In [11]:
import numpy as np
import pandas as pd
from sklearn.dummy import DummyRegressor
from sklearn.preprocessing import StandardScaler
from skforecast.recursive import ForecasterRecursive

# Global config
LEN_SERIES = 2000
STEPS = 100
RANDOM_STATE = 123


def _make_data(
    len_series: int = LEN_SERIES, 
    steps: int = STEPS, 
    random_state: int = RANDOM_STATE
) -> tuple[pd.Series, pd.DataFrame, pd.DataFrame]:
    """
    Create synthetic time series data for benchmarking.
    
    Parameters
    ----------
    len_series : int
        Length of the time series.
    steps : int
        Number of prediction steps to generate.
    random_state : int
        Random seed for reproducibility.
        
    Returns
    -------
    y : pandas Series
        Target time series.
    exog : pandas DataFrame
        Exogenous variables for training.
    exog_pred : pandas DataFrame
        Exogenous variables for prediction.
    
    """

    rng = np.random.default_rng(random_state)
    y = pd.Series(
        data=rng.normal(loc=20, scale=5, size=len_series),
        index=pd.date_range(start='2010-01-01', periods=len_series, freq='h'),
        name='y'
    )

    # Create exogenous variables with temporal features
    exog = pd.DataFrame(index=y.index)
    exog['day_of_week'] = exog.index.dayofweek
    exog['week_of_year'] = exog.index.isocalendar().week.astype(int)
    exog['month'] = exog.index.month

    # Create future exogenous variables for prediction
    exog_pred = pd.DataFrame(
        index=pd.date_range(
            start=exog.index.max() + pd.Timedelta(hours=1),
            periods=steps,
            freq='h'
        )
    )
    exog_pred['day_of_week'] = exog_pred.index.dayofweek
    exog_pred['week_of_year'] = exog_pred.index.isocalendar().week.astype(int)
    exog_pred['month'] = exog_pred.index.month

    return y, exog, exog_pred

In [12]:
y, exog, exog_pred = _make_data()
y_values = y.to_numpy()

forecaster = ForecasterRecursive(
    estimator=DummyRegressor(strategy='constant', constant=1.),
    lags=50,
    transformer_y=StandardScaler(),
    transformer_exog=StandardScaler(),
)

In [13]:
forecaster.fit(y=y, exog=exog, store_in_sample_residuals=True)

In [14]:
forecaster.in_sample_residuals_by_bin_.keys()

dict_keys([9])

In [15]:
forecaster.predict_bootstrapping(
            steps=STEPS,
            exog=exog_pred,
            n_boot=250,
            use_binned_residuals=False
        )

,pred_boot_0,pred_boot_1,pred_boot_2,pred_boot_3,pred_boot_4,pred_boot_5,pred_boot_6,pred_boot_7,pred_boot_8,pred_boot_9,...,pred_boot_240,pred_boot_241,pred_boot_242,pred_boot_243,pred_boot_244,pred_boot_245,pred_boot_246,pred_boot_247,pred_boot_248,pred_boot_249
2010-03-25 08:00:00,16.897624,17.403016,19.873710,15.733748,27.091060,17.568912,28.812210,19.014944,26.167307,24.092815,...,26.667447,22.401269,22.903187,23.921234,15.350504,21.643637,17.826981,17.293312,14.580454,19.221395
2010-03-25 09:00:00,17.220626,18.439558,26.578329,29.839045,16.327543,23.925508,28.302781,29.156948,21.282525,16.232112,...,19.570523,14.679188,18.935333,24.828422,26.167024,25.542094,16.468583,18.020467,28.999028,8.209379
2010-03-25 10:00:00,9.008297,18.650353,16.523329,11.562663,18.569439,23.949004,17.906527,18.408968,22.400022,9.008297,...,17.582543,19.312508,9.489287,19.916691,16.792538,17.157031,14.057763,21.987478,20.558938,22.817803
2010-03-25 11:00:00,18.015563,18.737392,26.825540,18.528265,22.281484,18.417586,18.848502,16.750698,20.614940,16.592047,...,23.656650,16.576271,27.062137,16.356077,21.070471,17.711942,18.541652,22.027151,18.734052,17.344365
2010-03-25 12:00:00,17.846927,26.491828,15.259615,18.967037,14.580454,24.746832,17.675282,15.084381,24.092815,12.226635,...,24.129275,12.556650,17.501257,24.626166,15.992072,14.734200,21.688136,16.930499,20.292542,25.633500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2010-03-29 07:00:00,22.615565,16.415488,25.669747,23.706504,20.161293,22.159209,14.685272,22.041363,19.695890,20.754440,...,15.783945,16.914746,23.759006,19.107465,20.438270,20.283192,28.000822,20.950079,24.406080,22.201798
2010-03-29 08:00:00,20.701795,24.575591,28.805325,17.695246,21.510793,26.082052,24.656757,26.815218,22.625444,20.706366,...,18.901307,33.635040,21.657525,28.568077,25.911709,18.485641,19.592735,11.104157,19.932600,22.569195
2010-03-29 09:00:00,17.082722,20.924691,24.247243,21.510793,23.825294,13.049487,24.524053,26.221602,29.085588,22.369615,...,19.681036,15.702844,16.011064,19.358757,22.615565,17.012169,24.522450,17.616786,13.988893,16.232112
2010-03-29 10:00:00,30.203406,13.064149,15.083947,24.092815,25.889877,11.626669,19.932600,16.439177,21.356677,10.850327,...,28.599447,29.974146,7.835133,20.624580,12.465613,16.670002,15.589925,28.266759,26.094398,18.500243
